In [ ]:
#default_exp core

In [ ]:
#export
import jsonref,json
from fastcore.utils import *
from fastcore.foundation import *
from fastcore.meta import *
from urllib import request
import pprint,inspect
from inspect import signature,Parameter
from ghapi.build_lib import build_funcs, GH_DOC_URL
from ghapi.metadata import funcs

In [ ]:
#export
GH_HOST = "https://api.github.com"
_DOC_URL = GH_DOC_URL

In [ ]:
import string
_fmt = string.Formatter()

def stringfmt_names(s:str)->L:
    "Unique brace-delimited names in `s`"
    return L(_fmt.parse(s)).itemgot(1).unique().filter()

In [ ]:
stringfmt_names('/pulls/{pull_number}/reviews/{review_id}')

(#2) ['pull_number','review_id']

In [ ]:
def _call(self, *args, **kwargs):
    # TODO: post data
    for a,b in zip(args,self.params): kwargs[b]=a
    kw = {p:kwargs[p] for p in self.params}
    path = self.path.format(**kw)
    return self.client(path)

_self_param = [Parameter('self', Parameter.POSITIONAL_OR_KEYWORD)]

def _replace_sig(f, path_args, data_args):
    p = [Parameter(o,               kind=Parameter.POSITIONAL_OR_KEYWORD) for o in path_args]
    d = [Parameter(o, default=None, kind=Parameter.POSITIONAL_OR_KEYWORD) for o in data_args]
    f.__signature__ = signature(f).replace(parameters=_self_param+p+d)
    return f

In [ ]:
class GhVerb:
    __slots__ = 'path,verb,tag,name,summary,url,params,data,client,__call__,__doc__'.split(',')
    
    def __init__(self, path, verb, oper, summary, url, data, client):
        tag,name = oper.split('/')
        name = name.replace('-','_')
        params = filter_ex(stringfmt_names(path), in_(('owner','repo')), True)
        if params and not params[0]: breakpoint()
        f = _replace_sig(copy_func(_call), params, data)
        __doc__ = summary
        __call__ = MethodType(f, self)
        store_attr(self.__slots__)
    
    def _repr_markdown_(self):
        params = ','.join(self.params+self.data)
        return f"[{self.tag}/{self.name}]({_DOC_URL}{self.url})({params}): {self.summary}"
    __repr__ = _repr_markdown_

In [ ]:
class _GhVerbGroup(AttrDict):
    def __init__(self, verbs): super().__init__(**{o.name:o for o in verbs})
    def _repr_markdown_(self): return "\n".join(f'- [{k}]({_DOC_URL}{v.url})' for k,v in self.items())

In [ ]:
#export
class GhApi:
    def __init__(self, owner, repo, token):
        funcs_ = L(funcs).starmap(GhVerb, client=self)
        self._fs = {k:_GhVerbGroup(v) for k,v in groupby(funcs_, 'tag').items()}
        self._headers = { 'Authorization' : 'token ' + token,
                         'Accept': 'application/vnd.github.v3+json'}
        self.owner,self.repo = owner,repo
        self.repo_url = f"{GH_HOST}/repos/{owner}/{repo}"

    def __dir__(self): return super().__dir__() + list(self._fs)

    def __getattr__(self,k):
        if k in self._fs: return self._fs[k]
        raise AttributeError(k)

    def __call__(self, path, **data):
        "Call GitHub API `path`"
        path = f"{self.repo_url}{path}"
        return dict2obj(do_request(path, headers=self._headers, **data))

In [ ]:
api = GhApi('fastai', 'fastcore', os.environ['FASTRELEASE_TOKEN'])

In [ ]:
api.git.get_ref('heads/master')

- ref: refs/heads/master
- node_id: MDM6UmVmMjI1NDYwNTk5OnJlZnMvaGVhZHMvbWFzdGVy
- url: https://api.github.com/repos/fastai/fastcore/git/refs/heads/master
- object: 
  - sha: d06d540f07dd51e7ec89b59dd802871e284653e2
  - type: commit
  - url: https://api.github.com/repos/fastai/fastcore/git/commits/d06d540f07dd51e7ec89b59dd802871e284653e2

In [ ]:
api.repos.create_webhook

[repos/create_webhook](https://docs.github.com/rest/reference/repos#create-a-repository-webhook)(name,config,events,active): Create a repository webhook

In [ ]:
api.git

- [create_blob](https://docs.github.com/rest/reference/git#create-a-blob)
- [get_blob](https://docs.github.com/rest/reference/git#get-a-blob)
- [create_commit](https://docs.github.com/rest/reference/git#create-a-commit)
- [get_commit](https://docs.github.com/rest/reference/git#get-a-commit)
- [list_matching_refs](https://docs.github.com/rest/reference/git#list-matching-references)
- [get_ref](https://docs.github.com/rest/reference/git#get-a-reference)
- [create_ref](https://docs.github.com/rest/reference/git#create-a-reference)
- [update_ref](https://docs.github.com/rest/reference/git#update-a-reference)
- [delete_ref](https://docs.github.com/rest/reference/git#delete-a-reference)
- [create_tag](https://docs.github.com/rest/reference/git#create-a-tag-object)
- [get_tag](https://docs.github.com/rest/reference/git#get-a-tag)
- [create_tree](https://docs.github.com/rest/reference/git#create-a-tree)
- [get_tree](https://docs.github.com/rest/reference/git#get-a-tree)

In [ ]:
api.git.get_ref.path

'/git/ref/{ref}'

In [ ]:
api.issues.list_for_repo()[1]

- url: https://api.github.com/repos/fastai/fastcore/issues/214
- repository_url: https://api.github.com/repos/fastai/fastcore
- labels_url: https://api.github.com/repos/fastai/fastcore/issues/214/labels{/name}
- comments_url: https://api.github.com/repos/fastai/fastcore/issues/214/comments
- events_url: https://api.github.com/repos/fastai/fastcore/issues/214/events
- html_url: https://github.com/fastai/fastcore/issues/214
- id: 744955123
- node_id: MDU6SXNzdWU3NDQ5NTUxMjM=
- number: 214
- title: move `patch` and `patch_to` to `fastcore.basics`, and avoid clobbering existing symbols when patching
- user: 
  - login: jph00
  - id: 346999
  - node_id: MDQ6VXNlcjM0Njk5OQ==
  - avatar_url: https://avatars1.githubusercontent.com/u/346999?v=4
  - gravatar_id: 
  - url: https://api.github.com/users/jph00
  - html_url: https://github.com/jph00
  - followers_url: https://api.github.com/users/jph00/followers
  - following_url: https://api.github.com/users/jph00/following{/other_user}
  - gists_url: https://api.github.com/users/jph00/gists{/gist_id}
  - starred_url: https://api.github.com/users/jph00/starred{/owner}{/repo}
  - subscriptions_url: https://api.github.com/users/jph00/subscriptions
  - organizations_url: https://api.github.com/users/jph00/orgs
  - repos_url: https://api.github.com/users/jph00/repos
  - events_url: https://api.github.com/users/jph00/events{/privacy}
  - received_events_url: https://api.github.com/users/jph00/received_events
  - type: User
  - site_admin: False
- labels: 
  - 
    - id: 1710689103
    - node_id: MDU6TGFiZWwxNzEwNjg5MTAz
    - url: https://api.github.com/repos/fastai/fastcore/labels/enhancement
    - name: enhancement
    - color: a2eeef
    - default: True
    - description: New feature or request
- state: open
- locked: False
- assignee: None
- assignees: 

- milestone: None
- comments: 0
- created_at: 2020-11-17T17:53:14Z
- updated_at: 2020-11-17T17:53:14Z
- closed_at: None
- author_association: MEMBER
- active_lock_reason: None
- body: 
- performed_via_github_app: None

In [ ]:
def _tag_date(self, tag):
    try: tag_d = self.gh(f"git/ref/tags/{tag}")
    except HTTPError: raise Exception(f"Failed to find tag {tag}")
    commit_d = self.gh(tag_d.object.url, complete=True)
    self.commit_date = commit_d.committer.date
    return self.commit_date

In [ ]:
def release(self):
    "Tag and create a release in GitHub for the current version"
    run(f'git tag {ver}')
    run('git push --tags')
    run('git pull --tags')
    self.gh("releases", post=True, tag_name=ver, name=ver, body=notes)

## Export -

In [ ]:

#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted build_lib.ipynb.
Converted index.ipynb.
